In [ ]:
import pandas as pd
from nltk.corpus import comparative_sentences  #keyword comparison 생성 모듈

data = pd.read_csv('reviews_preprocessing.csv', engine='python')
data.head()

In [ ]:
#데이터 확인
len(data)

In [ ]:
#kw 사전 리스트 생성
### kw_list = comparative_sentences.keywords_readme()[5:]
### kw_str = '|'.join(kw_list)

#kw 사전 수작업으로 분류 후 다시 읽어오기
def call_kw():
    kw_list = pd.read_csv('kw_list.csv',engine='python')
    #칼럼 중에 'to_use'만 가져오기
    kw_list = list(kw_list['to_use'])
#     kw_str = '|'.join(kw_list)
    
    return kw_list

for kw in call_kw():
    print(kw)

In [ ]:
#CS / SS 라벨링
kw_list=call_kw()
def tokenizedNtagging(text):
    #kw로 분류
    for kw in kw_list:
        if kw.lower() in text:
            return 'CS'
        
    #pos로 분류    
    from nltk import word_tokenize, pos_tag
    # nltk로 토크나이즈, 포스태깅
    tokens = word_tokenize(text)
    pos_tokens = pos_tag(tokens)

    count = 0
    for tok, pos in pos_tokens:
        if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
            count+=1

    if count != 0:
        return 'CS'
    else:
        return 'SS'            

In [ ]:
#모델 사전 호출 함수
def call_model():
    df=pd.read_csv('product_list.csv', engine='python')
    model_ls=list(df['product'])
    model_reg='( '+' | '.join(model_ls)+' )'
    return model_reg

In [ ]:
#리뷰에서 언급된 모델명 탐색
import re
import numpy as np

model_reg = call_model()
def targetmodel_read(text):
    read_ls = re.findall(model_reg,text)
    read_ls = list(set(read_ls))
    if read_ls :
        return ','.join(read_ls).replace(' ','')
    else:
        return np.nan

In [ ]:
#자기 자신을 언급한 제품 삭제
def targetmodel_del(df):
    new_df = pd.DataFrame(columns = df.columns)
    for idx in df.index:
        if not df.loc[idx,'target'] in df.loc[idx, 'product'].lower():
            new_df= pd.concat([new_df,pd.DataFrame(df.loc[idx]).T])
        else:
            print('제거 : ',df.loc[idx, 'product'], df.loc[idx,'target'])
    return new_df

In [ ]:
# #target 명을 아주 길게길게 만들어줍시다.
# def targetmodel_full(df,prod_ls):
#     def check_model(target):
#         new_model = []
#         model_ls = target.split(',')
#         for model in model_ls:
#             for prod in prod_ls:
#                 if model in prod.lower():
#                     new_model.append(prod)
                    
#         if new_model:
#             print(new_model)
#             return ','.join(new_model)
#         else:
#             return target
    
#     df['target']=df['target'].apply(check_model)
#     return df

In [ ]:
#csss 라벨링, 타겟 제품 검색 모델링
def make_sscs(data):
    product_ls=list(set(list(data['product'])))
    
    data['CS_SS']=data['review']
    data['target']=data['review']
    data['CS_SS']=data['CS_SS'].apply(tokenizedNtagging)
    print('CS_SS 라벨링 끝')
    data_cs=data[data['CS_SS']=='CS']
    data_ss=data[data['CS_SS']=='SS']
    data_ss['target']=data_ss['target'].apply(lambda x:'NaN')
    data_cs['target']=data_cs['target'].apply(targetmodel_read)
    print('모델명 탐색 끝')
    data_cs=data_cs.dropna()
    data_cs=targetmodel_del(data_cs)
    print('자기 자신 호출 CS 제거')
#     data_cs=targetmodel_full(data_cs,product_ls)
#     print('target 명 길게 늘리기 완료')
    new_data=pd.concat([data_cs,data_ss])
    return new_data

In [ ]:
df_t = make_sscs(data)
df_t.head()

In [ ]:
len(df_t[df_t.CS_SS=='CS'])

In [ ]:
len(df_t[df_t.CS_SS=='SS'])

In [ ]:
df_t.to_csv('labelingSSCS.csv',index=False, encoding='utf-8')

======================== 테스트 영역 ================================================